In [2]:
#standard library
import pandas as pd
import numpy as np

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#preprocessing
from sklearn.preprocessing import Imputer, StandardScaler

#import models
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

#grid search
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#classification imbalance
from imblearn.over_sampling import SMOTE

# model evaluation
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score , roc_curve, auc

In [ ]:
df = pd.read_csv('./all/health-diagnostics-train.csv')
test = pd.read_csv('./all/health-diagnostics-test.csv')

In [ ]:
df.replace('#NULL!',np.nan, inplace=True)
df.iloc[:, 0:(len(df.columns)-1)] = df.select_dtypes(include='object').apply(pd.to_numeric)
df.dropna(inplace=True)
test.replace('#NULL!',np.nan, inplace=True)
test = test.select_dtypes(include='object').apply(pd.to_numeric)
imp = Imputer(strategy='most_frequent')
imp.fit(test)
F_test = imp.transform(test)

In [ ]:
X = df.drop('target',axis=1)
y=df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69, stratify=y)

# Grid Search

## Logistic Regression

In [ ]:
gsc = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid={
        'class_weight':[{0:0.1,1:0.99},'balanced'],
        'C':[0.01,0.1,1.0,2.5,10,30,50]
    },
    scoring='precision',
    cv=10,
    refit=False,
    verbose=3,
    n_jobs=-1
)
gsc.fit(X, y)

In [ ]:
gsc.best_params_

In [ ]:
scores = cross_val_score(LogisticRegression(C=0.1, class_weight={0:0.01,1:0.99}, solver='sag'), X, y ,cv=50, scoring='precision', n_jobs=-1, verbose=3)

In [ ]:
np.max(scores),np.min(scores),np.mean(scores),np.std(scores)

In [ ]:
sns.lineplot(x=np.linspace(0,len(scores),len(scores)),y=scores)

In [ ]:
{'C': 0.1, 'class_weight': {0: 0.1, 1: 0.99}}

In [ ]:
gsc.best_score_

# SGDClassifier

In [ ]:
gsc = GridSearchCV(
    estimator=SGDClassifier(class_weight={0:0.01,1:0.99}),
    param_grid={
    'alpha': [0], 
    'loss': ['log'], 
    'penalty': ['l2'],
    'n_jobs': [-1]
    },
    scoring='precision',
    cv=10,
    refit=False,
    verbose=3,
    n_jobs=-1
)
gsc.fit(X, y)

In [ ]:
gsc.best_params_

In [ ]:
gsc.best_score_

In [ ]:
scores = cross_val_score(SGDClassifier(loss='log', class_weight={0:0.01,1:0.99}, alpha=0.0015), X, y ,cv=10, scoring='roc_auc', n_jobs=-1)

In [ ]:
sgd = SGDClassifier(loss='log', class_weight={0:0.01,1:0.99}, random_state=123)
sgd.fit(X,y)
y_pred = sgd.predict(F_test)
predictions = pd.Series(y_pred)

In [ ]:
predictions = predictions.reset_index()
predictions.columns = ['index','target']
predictions.to_csv('submissions.csv', index=False)

In [ ]:
np.mean(scores)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101, stratify=y)
sgd = SGDClassifier(loss='log', class_weight={0:0.01,1:0.99})
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
pd.crosstab(y_test, y_pred)

In [ ]:
scores

In [ ]:
sns.lineplot(x=alphas,y=scores)

In [ ]:
np.max(scores),np.min(scores),np.mean(scores),np.std(scores)

In [ ]:
sns.lineplot(x=np.linspace(0,len(scores),len(scores)),y=scores)

# SVM

In [ ]:
rand_svc = RandomizedSearchCV(
    SVC(),
    param_distributions={
        'class_weight':[{0:0.1,1:0.99},'balanced'],
        'C':[1,10,25,50,100],
        'kernel':['linear'],
        'degree':[1]
    },
    scoring='precision',
    n_iter=10,
    refit=False,
    cv=5,
    n_jobs=-1,
    verbose=3
)
rand_svc.fit(X,y)

In [ ]:
param_test1 = {
    'C':[1,2,3,4,5],
    'gamma':[0.001, 0.01,0.5,1,2,5,'auto']
}

grid = GridSearchCV(SVC(kernel='linear', degree=1, class_weight={0:0.01,1:0.99}), 
                    param_grid = param_test1, 
                    scoring='precision',
                    n_jobs=-1,
                    refit=False, 
                    cv=5,
                    verbose=3)

grid.fit(X,y)
grid.grid_scores_, grid.best_params_, grid.best_score_

In [ ]:
scores = cross_val_score(SVC(C=1, gamma=0.001, kernel='linear', degree=1, class_weight={0:0.01,1:0.99}), X, y ,cv=50, scoring='roc_auc', n_jobs=-1, verbose=3)

In [ ]:
np.max(scores),np.min(scores),np.mean(scores),np.std(scores)

In [ ]:
sns.lineplot(x=np.linspace(0,len(scores),len(scores)),y=scores)

## XGBoost

XGBoost + SMOTE

Top Performer: 

Kaggle - 0.74724

FP - 25
TP - 10
FN - 2

In [ ]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=)
modelfit(xgb1)

In [ ]:
ratio = float(len(y_train_sm) - np.sum(y))/ np.sum(y_train_sm)

In [ ]:
[x for x in range(3,10,2)]

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=122, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=ratio, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train_sm,y_train_sm)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test2 = {
 'max_depth':[8,9,10,11,15],
 'min_child_weight':[8,9,10,11,15]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=122, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=ratio, seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train_sm,y_train_sm)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test3 = {
 'min_child_weight':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=122, max_depth=11,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=ratio, seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train_sm,y_train_sm)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test4 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=122, max_depth=11,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=ratio, seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train_sm,y_train_sm)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test5 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=122, max_depth=11,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=ratio, seed=27), 
 param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train_sm,y_train_sm)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test6 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=122, max_depth=11,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=ratio, seed=27), 
 param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train_sm,y_train_sm)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
xgb2 = XGBClassifier(
 learning_rate =0.01,
 n_estimators=122,
 max_depth=11,
 min_child_weight=1,
 gamma=0.1,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=ratio,
 seed=27)
modelfit(xgb2)

In [ ]:
parameters = {
              'learning_rate': [0.01,0.05,0.1,0.2],
              'max_depth': [11],
              'n_estimators': [122],
              'gamma':[0.1],
              'min_child_weight':[1,2,3],
              'scale_pos_weight':[1,ratio],
              'reg_alpha':[1e-5, 1e-2, 0.1]
             }


clf = GridSearchCV(XGBClassifier(), 
                   parameters, 
                   n_jobs=3, 
                   cv=3,
                   scoring='roc_auc',
                   verbose=1, 
                   refit=True)

clf.fit(X_train_sm, y_train_sm)

# Random Forest

In [ ]:
param_grid = {
    'n_estimators':[25,75],
    'max_depth':[2,5],
    'min_samples_leaf':[0.001, 0.003],
    'min_samples_split':[0.01],
    'min_impurity_decrease':[0.01,0.02],
    'class_weight':['balanced',{0:0.01,1:0.99}],
    max_features:['auto']
}
grid = GridSearchCV(RandomForestClassifier(), param_grid, scoring='precision',n_jobs=4,refit=False, cv=5, verbose=3)

grid.fit(X_train,y_train)
grid.grid_scores_, grid.best_params_, grid.best_score_

In [ ]:
rfc = RandomForestClassifier(
    n_estimators=75,
    max_depth=2,
    min_impurity_decrease=0.02,
    min_samples_leaf=0.003,
    min_samples_split=0.01,
    class_weight={0:0.01,1:0.99},
    max_features='auto'
)
scores = cross_val_score(rfc, X, y ,cv=10, scoring='roc_auc', n_jobs=-1, verbose=3)

In [ ]:
np.max(scores),np.min(scores),np.mean(scores),np.std(scores)

In [ ]:
sns.lineplot(x=np.linspace(0,len(scores),len(scores)),y=scores)